<i>Copyright (c) Microsoft Corporation. All rights reserved.</i>

<i>Licensed under the MIT License.</i>

# Benchmark with Movielens dataset

This illustrative comparison applies to collaborative filtering algorithms available in this repository such as Spark ALS, Surprise SVD, SAR and others using the Movielens dataset. These algorithms are usable in a variety of recommendation tasks, including product or news recommendations.

The main purpose of this notebook is not to produce comprehensive benchmarking results on multiple datasets. Rather, it is intended to illustrate on how one could evaluate different recommender algorithms using tools in this repository.

## Experimentation setup:

* Objective
  * To compare how each collaborative filtering algorithm perform in predicting ratings and recommending relevant items.

* Environment
  * The comparison is run on a [Azure Data Science Virtual Machine](https://azure.microsoft.com/en-us/services/virtual-machines/data-science-virtual-machines/). 
  * The virtual machine size is Standard NC6 (6 vcpus, 55 GB memory, 1K80 GPU).
  * It should be noted that the single node DSVM is not supposed to run scalable benchmarking analysis. Either scaling up or out the computing instances is necessary to run the benchmarking in an run-time efficient way without any memory issue.
  * **NOTE ABOUT THE DEPENDENCIES TO INSTALL**: This notebook uses CPU, GPU and PySpark algorithms, so make sure you install the `full environment` as detailed in the [SETUP.md](../../SETUP.md). 
  
* Datasets
  * [Movielens 100K](https://grouplens.org/datasets/movielens/100k/).
  * [Movielens 1M](https://grouplens.org/datasets/movielens/1m/).

* Data split
  * The data is split into train and test sets.
  * The split ratios are 75-25 for train and test datasets.
  * The splitting is stratified based on items. 

* Model training
  * A recommendation model is trained by using each of the collaborative filtering algorithms. 
  * Empirical parameter values reported [here](http://mymedialite.net/examples/datasets.html) are used in this notebook.  More exhaustive hyper parameter tuning would be required to further optimize results.

* Evaluation metrics
  * Ranking metrics:
    * Precision@k.
    * Recall@k.
    * Normalized discounted cumulative gain@k (NDCG@k).
    * Mean-average-precision (MAP). 
    * In the evaluation metrics above, k = 10. 
  * Rating metrics:
    * Root mean squared error (RMSE).
    * Mean average error (MAE).
    * R squared.
    * Explained variance.
  * Run time performance
    * Elapsed for training a model and using a model for predicting/recommending k items. 
    * The time may vary across different machines. 

## Globals settings

In [1]:
import sys
import os
import json
import pandas as pd
import numpy as np
import seaborn as sns
import pyspark
import torch
import fastai
import tensorflow as tf
tf.get_logger().setLevel('ERROR') # only show error messages
import surprise

from recommenders.utils.general_utils import get_number_processors
from recommenders.utils.gpu_utils import get_cuda_version, get_cudnn_version
from recommenders.datasets import movielens
from recommenders.datasets.python_splitters import python_stratified_split
from recommenders.models.fastai.fastai_utils import hide_fastai_progress_bar

from benchmark_utils import * 

print("System version: {}".format(sys.version))
print("Pandas version: {}".format(pd.__version__))
print("PySpark version: {}".format(pyspark.__version__))
print("Surprise version: {}".format(surprise.__version__))
print("PyTorch version: {}".format(torch.__version__))
print("Fast AI version: {}".format(fastai.__version__))
print("Cornac version: {}".format(cornac.__version__))
print("Tensorflow version: {}".format(tf.__version__))
print("CUDA version: {}".format(get_cuda_version()))
print("CuDNN version: {}".format(get_cudnn_version()))
n_cores = get_number_processors()
print("Number of cores: {}".format(n_cores))

%load_ext autoreload
%autoreload 2

/anaconda/envs/azureml_py38/lib/python3.8/site-packages/dask/dataframe/utils.py:367: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  _numeric_index_types = (pd.Int64Index, pd.Float64Index, pd.UInt64Index)
/anaconda/envs/azureml_py38/lib/python3.8/site-packages/dask/dataframe/utils.py:367: FutureWarning: pandas.Float64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  _numeric_index_types = (pd.Int64Index, pd.Float64Index, pd.UInt64Index)
/anaconda/envs/azureml_py38/lib/python3.8/site-packages/dask/dataframe/utils.py:367: FutureWarning: pandas.UInt64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  _numeric_index_types = (pd.Int64Index, pd.Float64Index, pd.UInt64Index)


System version: 3.8.5 (default, Sep  4 2020, 07:30:14) 
[GCC 7.3.0]
Pandas version: 1.4.2
PySpark version: 3.2.1
Surprise version: 1.1.1
PyTorch version: 1.9.0+cu111
Fast AI version: 1.0.61
Cornac version: 1.14.2
Tensorflow version: 2.7.3
CUDA version: No CUDA in this machine
CuDNN version: 8.2.1
Number of cores: 6


## Parameters

In [2]:
# Hide fastai progress bar
hide_fastai_progress_bar()

In [3]:
# fix random seeds to make sure out runs are reproducible
np.random.seed(SEED)
torch.manual_seed(SEED)
torch.cuda.manual_seed_all(SEED)

In [4]:
environments = {
    "als": "pyspark",
    "sar": "python_cpu",
    "svd": "python_cpu",
    "fastai": "python_gpu",
    "ncf": "python_gpu",
    "bpr": "python_cpu",
    "bivae": "python_gpu",
    "lightgcn": "python_gpu",
}

metrics = {
    "als": ["rating", "ranking"],
    "sar": ["ranking"],
    "svd": ["rating", "ranking"],
    "fastai": ["rating", "ranking"],
    "ncf": ["ranking"],
    "bpr": ["ranking"],
    "bivae": ["ranking"],
    "lightgcn": ["ranking"]
}

Algorithm parameters

In [5]:
als_params = {
    "rank": 10,
    "maxIter": 15,
    "implicitPrefs": False,
    "alpha": 0.1,
    "regParam": 0.05,
    "coldStartStrategy": "drop",
    "nonnegative": False,
    "userCol": DEFAULT_USER_COL,
    "itemCol": DEFAULT_ITEM_COL,
    "ratingCol": DEFAULT_RATING_COL,
}

sar_params = {
    "similarity_type": "jaccard",
    "time_decay_coefficient": 30,
    "time_now": None,
    "timedecay_formula": True,
    "col_user": DEFAULT_USER_COL,
    "col_item": DEFAULT_ITEM_COL,
    "col_rating": DEFAULT_RATING_COL,
    "col_timestamp": DEFAULT_TIMESTAMP_COL,
}

svd_params = {
    "n_factors": 150,
    "n_epochs": 15,
    "lr_all": 0.005,
    "reg_all": 0.02,
    "random_state": SEED,
    "verbose": False
}

fastai_params = {
    "n_factors": 40, 
    "y_range": [0,5.5], 
    "wd": 1e-1,
    "max_lr": 5e-3,
    "epochs": 15
}

ncf_params = {
    "model_type": "NeuMF",
    "n_factors": 4,
    "layer_sizes": [16, 8, 4],
    "n_epochs": 15,
    "batch_size": 1024,
    "learning_rate": 1e-3,
    "verbose": 10
}

bpr_params = {
    "k": 200,
    "max_iter": 200,
    "learning_rate": 0.01,
    "lambda_reg": 1e-3,
    "seed": SEED,
    "verbose": False
}

bivae_params = {
    "k": 100,
    "encoder_structure": [200],
    "act_fn": "tanh",
    "likelihood": "pois",
    "n_epochs": 500,
    "batch_size": 1024,
    "learning_rate": 0.001,
    "seed": SEED,
    "use_gpu": True,
    "verbose": False
}

lightgcn_param = {
    "yaml_file": os.path.join("..","..", "recommenders","models","deeprec", "config", "lightgcn.yaml"),
    "n_layers": 3,
    "batch_size": 1024,
    "epochs": 15,
    "learning_rate": 0.005,
    "eval_epoch": 5,
    "top_k": DEFAULT_K,
}

params = {
    "als": als_params,
    "sar": sar_params,
    "svd": svd_params,
    "fastai": fastai_params,
    "ncf": ncf_params,
    "bpr": bpr_params,
    "bivae": bivae_params,
    "lightgcn": lightgcn_param,
}

In [6]:
prepare_training_data = {
    "als": prepare_training_als,
    "sar": prepare_training_sar,
    "svd": prepare_training_svd,
    "fastai": prepare_training_fastai,
    "ncf": prepare_training_ncf,
    "bpr": prepare_training_cornac,
    "bivae": prepare_training_cornac,
    "lightgcn": prepare_training_lightgcn,
}

In [7]:
prepare_metrics_data = {
    "als": lambda train, test: prepare_metrics_als(train, test),
    "fastai": lambda train, test: prepare_metrics_fastai(train, test),    
}

In [8]:
trainer = {
    "als": lambda params, data: train_als(params, data),
    "svd": lambda params, data: train_svd(params, data),
    "sar": lambda params, data: train_sar(params, data), 
    "fastai": lambda params, data: train_fastai(params, data),
    "ncf": lambda params, data: train_ncf(params, data),
    "bpr": lambda params, data: train_bpr(params, data),
    "bivae": lambda params, data: train_bivae(params, data),
    "lightgcn": lambda params, data: train_lightgcn(params, data),
}

In [9]:
rating_predictor = {
    "als": lambda model, test: predict_als(model, test),
    "svd": lambda model, test: predict_svd(model, test),
    "fastai": lambda model, test: predict_fastai(model, test),
}

In [10]:
ranking_predictor = {
    "als": lambda model, test, train: recommend_k_als(model, test, train),
    "sar": lambda model, test, train: recommend_k_sar(model, test, train),
    "svd": lambda model, test, train: recommend_k_svd(model, test, train),
    "fastai": lambda model, test, train: recommend_k_fastai(model, test, train),
    "ncf": lambda model, test, train: recommend_k_ncf(model, test, train),
    "bpr": lambda model, test, train: recommend_k_cornac(model, test, train),
    "bivae": lambda model, test, train: recommend_k_cornac(model, test, train),
    "lightgcn": lambda model, test, train: recommend_k_lightgcn(model, test, train),
}

In [11]:
rating_evaluator = {
    "als": lambda test, predictions: rating_metrics_pyspark(test, predictions),
    "svd": lambda test, predictions: rating_metrics_python(test, predictions),
    "fastai": lambda test, predictions: rating_metrics_python(test, predictions)
}
    
    
ranking_evaluator = {
    "als": lambda test, predictions, k: ranking_metrics_pyspark(test, predictions, k),
    "sar": lambda test, predictions, k: ranking_metrics_python(test, predictions, k),
    "svd": lambda test, predictions, k: ranking_metrics_python(test, predictions, k),
    "fastai": lambda test, predictions, k: ranking_metrics_python(test, predictions, k),
    "ncf": lambda test, predictions, k: ranking_metrics_python(test, predictions, k),
    "bpr": lambda test, predictions, k: ranking_metrics_python(test, predictions, k),
    "bivae": lambda test, predictions, k: ranking_metrics_python(test, predictions, k),
    "lightgcn": lambda test, predictions, k: ranking_metrics_python(test, predictions, k),
}

In [12]:
def generate_summary(data, algo, k, train_time, time_rating, rating_metrics, time_ranking, ranking_metrics):
    summary = {"Data": data, "Algo": algo, "K": k, "Train time (s)": train_time, "Predicting time (s)": time_rating, "Recommending time (s)": time_ranking}
    if rating_metrics is None:
        rating_metrics = {
            "RMSE": np.nan,
            "MAE": np.nan,
            "R2": np.nan,
            "Explained Variance": np.nan,
        }
    if ranking_metrics is None:
        ranking_metrics = {
            "MAP": np.nan,
            "nDCG@k": np.nan,
            "Precision@k": np.nan,
            "Recall@k": np.nan,
        }
    summary.update(rating_metrics)
    summary.update(ranking_metrics)
    return summary

## Benchmark loop

In [13]:
data_sizes = ["100k", "1m"] # Movielens data size: 100k, 1m, 10m, or 20m
algorithms = ["als", "svd", "sar", "ncf", "fastai", "bpr", "bivae", "lightgcn"]


In [14]:
%%time

# For each data size and each algorithm, a recommender is evaluated. 
cols = ["Data", "Algo", "K", "Train time (s)", "Predicting time (s)", "RMSE", "MAE", "R2", "Explained Variance", "Recommending time (s)", "MAP", "nDCG@k", "Precision@k", "Recall@k"]
df_results = pd.DataFrame(columns=cols)

for data_size in data_sizes:
    # Load the dataset
    df = movielens.load_pandas_df(
        size=data_size,
        header=[DEFAULT_USER_COL, DEFAULT_ITEM_COL, DEFAULT_RATING_COL, DEFAULT_TIMESTAMP_COL]
    )
    print("Size of Movielens {}: {}".format(data_size, df.shape))
    
    # Split the dataset
    df_train, df_test = python_stratified_split(df,
                                                ratio=0.75, 
                                                min_rating=1, 
                                                filter_by="item", 
                                                col_user=DEFAULT_USER_COL, 
                                                col_item=DEFAULT_ITEM_COL
                                                )
   
    # Loop through the algos
    for algo in algorithms:
        print(f"\nComputing {algo} algorithm on Movielens {data_size}")
          
        # Data prep for training set
        train = prepare_training_data.get(algo, lambda x,y:(x,y))(df_train, df_test)
        
        # Get model parameters
        model_params = params[algo]
          
        # Train the model
        model, time_train = trainer[algo](model_params, train)
        print(f"Training time: {time_train}s")
                
        # Predict and evaluate
        train, test = prepare_metrics_data.get(algo, lambda x,y:(x,y))(df_train, df_test)
        
        if "rating" in metrics[algo]:   
            # Predict for rating
            preds, time_rating = rating_predictor[algo](model, test)
            print(f"Rating prediction time: {time_rating}s")
            
            # Evaluate for rating
            ratings = rating_evaluator[algo](test, preds)
        else:
            ratings = None
            time_rating = np.nan
        
        if "ranking" in metrics[algo]:
            # Predict for ranking
            top_k_scores, time_ranking = ranking_predictor[algo](model, test, train)
            print(f"Ranking prediction time: {time_ranking}s")
            
            # Evaluate for rating
            rankings = ranking_evaluator[algo](test, top_k_scores, DEFAULT_K)
        else:
            rankings = None
            time_ranking = np.nan
            
        # Record results
        summary = generate_summary(data_size, algo, DEFAULT_K, time_train, time_rating, ratings, time_ranking, rankings)
        df_results.loc[df_results.shape[0] + 1] = summary
        
print("\nComputation finished")

INFO:recommenders.datasets.download_utils:Downloading https://files.grouplens.org/datasets/movielens/ml-100k.zip
100%|██████████| 4.81k/4.81k [00:00<00:00, 18.9kKB/s]


Size of Movielens 100k: (100000, 4)

Computing als algorithm on Movielens 100k
Training time: 7.2316s
Rating prediction time: 0.0861s


/anaconda/envs/azureml_py38/lib/python3.8/site-packages/pyspark/sql/context.py:125: FutureWarning: Deprecated in 3.0.0. Use SparkSession.builder.getOrCreate() instead.
  warnings.warn(


Ranking prediction time: 0.0869s

Computing svd algorithm on Movielens 100k
Training time: 4.3036s
Rating prediction time: 0.2510s
Ranking prediction time: 14.3849s


INFO:root:Collecting user affinity matrix
INFO:root:Calculating time-decayed affinities
INFO:root:Creating index columns
INFO:root:Building user affinity sparse matrix
INFO:root:Calculating item co-occurrence


...

Ranking prediction time: 2.4481s

Computing ncf algorithm on Movielens 1m


INFO:recommenders.models.ncf.dataset:Indexing ./df_train.csv ...
/anaconda/envs/azureml_py38/lib/python3.8/site-packages/tensorflow/python/keras/engine/base_layer_v1.py:1694: UserWarning: `layer.apply` is deprecated and will be removed in a future version. Please use `layer.__call__` method instead.
  warnings.warn('`layer.apply` is deprecated and '
INFO:recommenders.models.ncf.ncf_singlenode:Epoch 10 [45.82s]: train_loss = 0.296152 


Training time: 686.9924s
Ranking prediction time: 51.7218s

Computing fastai algorithm on Movielens 1m
Training time: 680.4962s
Rating prediction time: 0.3984s
Ranking prediction time: 47.4308s

Computing bpr algorithm on Movielens 1m
Training time: 65.8879s
Ranking prediction time: 50.5751s

Computing bivae algorithm on Movielens 1m
Training time: 219.3037s
Ranking prediction time: 28.6644s

Computing lightgcn algorithm on Movielens 1m


/anaconda/envs/azureml_py38/lib/python3.8/site-packages/recommenders/models/deeprec/DataModel/ImplicitCF.py:73: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = train if test is None else train.append(test)


Already create adjacency matrix.
Already normalize adjacency matrix.
Using xavier initialization.
Epoch 1 (train)407.1s: train loss = 0.34797 = (mf)0.34738 + (embed)0.00059
Epoch 2 (train)406.8s: train loss = 0.27672 = (mf)0.27538 + (embed)0.00133
Epoch 3 (train)406.3s: train loss = 0.22676 = (mf)0.22449 + (embed)0.00227
Epoch 4 (train)406.4s: train loss = 0.20416 = (mf)0.20120 + (embed)0.00296
Epoch 5 (train)406.8s + (eval)3.8s: train loss = 0.18869 = (mf)0.18510 + (embed)0.00359, recall = 0.12050, ndcg = 0.36984, precision = 0.33454, map = 0.07163
Epoch 6 (train)406.1s: train loss = 0.17532 = (mf)0.17113 + (embed)0.00420
Epoch 7 (train)410.3s: train loss = 0.16492 = (mf)0.16014 + (embed)0.00478
Epoch 8 (train)411.4s: train loss = 0.15507 = (mf)0.14971 + (embed)0.00536
Epoch 9 (train)410.9s: train loss = 0.14920 = (mf)0.14328 + (embed)0.00592
Epoch 10 (train)407.1s + (eval)3.8s: train loss = 0.14199 = (mf)0.13554 + (embed)0.00645, recall = 0.13939, ndcg = 0.40932, precision = 0.37111,

## Results

In [15]:
df_results

,Data,Algo,K,Train time (s),Predicting time (s),RMSE,MAE,R2,Explained Variance,Recommending time (s),MAP,nDCG@k,Precision@k,Recall@k
1,100k,als,10,7.2316,0.0861,0.966505,0.752611,0.253207,0.249371,0.0869,0.006217,0.053377,0.054825,0.020420
2,100k,svd,10,4.3036,0.2510,0.938681,0.742690,0.291967,0.291971,14.3849,0.012873,0.095930,0.091198,0.032783
3,100k,sar,10,0.3554,NaN,NaN,NaN,NaN,NaN,0.0881,0.113028,0.388321,0.333828,0.183179
4,100k,ncf,10,52.1639,NaN,NaN,NaN,NaN,NaN,3.6720,0.105227,0.394226,0.349311,0.179064
5,100k,fastai,10,78.5280,0.0444,0.941035,0.742262,0.288411,0.290805,3.0010,0.025521,0.147204,0.130753,0.054545
6,100k,bpr,10,6.1744,NaN,NaN,NaN,NaN,NaN,3.9153,0.129946,0.437411,0.383669,0.209318
7,100k,bivae,10,22.0689,NaN,NaN,NaN,NaN,NaN,1.5642,0.147895,0.478870,0.415164,0.221764
8,100k,lightgcn,10,65.8775,NaN,NaN,NaN,NaN,NaN,0.0693,0.118297,0.406401,0.352174,0.194290
9,1m,als,10,5.6175,0.0266,0.860085,0.679199,0.412694,0.406956,0.0620,0.002011,0.023573,0.029551,0.009647
10,1m,svd,10,43.7867,3.1679,0.883017,0.695366,0.374910,0.374911,198.7200,0.008828,0.089320,0.082856,0.021582
